# Semantic Segmentation of Water using U-Net
# Part 8 - Model Testing

In [4]:
%matplotlib inline
import pickle
import glob

from unetlib.metrics import BinaryMeanIoU
from unetlib.model import UNet_BN
from unetlib.preprocessing import make_dataframes_for_flow, make_img_msk_flows
import unetlib.visualisation as vs

In [5]:
# Imagery directories
nwpu_data_dir = 'nwpu_lake_images/data/'
nwpu_mask_dir = 'nwpu_lake_images/masks/'

In [6]:
out_dir = 'model_outputs/'

## Retrain Model

Now that I've selected my best model I can test it to see how it performs on the unseen test data to get a final evaluation. I no longer have need for the validation set so I can now retrain the model using the full training set i.e. no validation split.

This *should* help with model generalisation as it will be trained on more images.

In [4]:
# Create and train model with no validation
opt = 'RMSProp'
lr = 0.01
batch_size = 32


runs = 1
losses, ious = [], []

for i in range(runs):
    model = UNet_BN(n_filters=64, n_blocks=4, bn_pos='before', model_name='final')
    compile_model(model, opt, lr)
    
    # Generate filename for output files
    opt_conf = model.optimizer.get_config()
    o_name = opt_conf['name']
    o_lr = f"{tf.keras.backend.eval(opt_conf['learning_rate']):.3f}"
    base_fn = os.path.join(out_dir, f"{model.name}_{o_name}_lr{o_lr}_bs{batch_size}{{}}")
    
    
    ## Configure callbacks
    # Checkpointer
    checkpointer = tf.keras.callbacks.ModelCheckpoint(base_fn.format('.weights.h5'),
                                                      save_best_only=True,
                                                      save_weights_only=True
                                                      )
    # Timer
    timer = TrainingTimer()
    
    # Early stopping
    early_stop = EarlyStopping(patience=50)
    
    
    callbacks=[early_stop, checkpointer, timer]
    
    # Split the test/train data
    (train_img_df, train_msk_df,
     test_img_df, test_msk_df) = make_dataframes_for_flow(nwpu_data_dir,
                                                          nwpu_mask_dir,
                                                          test_size=0.25,
                                                          random_state=42
                                                          )

    # Split the training data into train and validation generators
    # with augmentation applied to the training data only
    aug_dict = {'rotation_range': 90,
                'horizontal_flip': True,
                'vertical_flip': True,
                'width_shift_range': 0.15,
                'height_shift_range': 0.15,
                'zoom_range': 0.25
                }

    (train_gen, train_fps) = make_img_msk_flows(train_img_df, train_msk_df,
                                                nwpu_data_dir, nwpu_mask_dir,
                                                val_split=0, rescale=1 / 255.,
                                                aug_dict=aug_dict,
                                                batch_size=batch_size
                                               )

    # Compute steps per epoch
    train_steps = int(np.ceil(len(train_fps) / batch_size))


    # Train the model
    history = model.fit(train_gen, epochs=epochs, steps_per_epoch=train_steps,
                        callbacks=callbacks
                       )

## Evaluate on Test Data

## Visualise Predictions